In [100]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [45]:
# Load the data
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# Assign X and Y from training data
X_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"].values

# Normalize the data for better performance
X_train = X_train / 255.0
test = test / 255.0

# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)

In [86]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu", strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu", strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, kernel_size=(4, 4), activation="relu", strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_103 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_42 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_43 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_44 (Batc (None, 12, 12, 32)        128       
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 6, 6, 32)        

In [99]:
# CREATE MORE IMAGES VIA DATA AUGMENTATION
batch_size = 150
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.10,
        width_shift_range=0.1,
        height_shift_range=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10)
batches = datagen.flow(X_train, y_train, batch_size=batch_size)
val_batches = datagen.flow(X_val, y_val, batch_size=batch_size)

In [101]:
epochs = 50
# Set a learning rate annealer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.00001)
history = model.fit(
    batches,
    epochs=epochs,
    steps_per_epoch=X_train.shape[0] // batch_size,
    validation_data=val_batches,
    callbacks=[reduce_lr])

Epoch 1/50
79/79 [==============================] - 17s 220ms/step - loss: 0.0257 - accuracy: 0.9930 - val_loss: 0.0380 - val_accuracy: 0.9886
Epoch 2/50
79/79 [==============================] - 18s 223ms/step - loss: 0.0889 - accuracy: 0.9751 - val_loss: 0.0724 - val_accuracy: 0.9788
Epoch 3/50
79/79 [==============================] - 18s 223ms/step - loss: 0.0369 - accuracy: 0.9900 - val_loss: 0.0352 - val_accuracy: 0.9894
Epoch 4/50
79/79 [==============================] - 18s 222ms/step - loss: 0.0268 - accuracy: 0.9921 - val_loss: 0.0189 - val_accuracy: 0.9924
Epoch 5/50
79/79 [==============================] - 20s 248ms/step - loss: 0.0387 - accuracy: 0.9876 - val_loss: 0.0459 - val_accuracy: 0.9871
Epoch 6/50
79/79 [==============================] - 21s 266ms/step - loss: 0.0268 - accuracy: 0.9924 - val_loss: 0.0261 - val_accuracy: 0.9917
Epoch 7/50
79/79 [==============================] - ETA: 0s - loss: 0.0191 - accuracy: 0.9942
Epoch 00007: ReduceLROnPlateau reducing learning

79/79 [==============================] - 18s 234ms/step - loss: 0.0073 - accuracy: 0.9978 - val_loss: 0.0062 - val_accuracy: 0.9985


In [102]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)